#### This is a basic example code on pre installed data see the fitting of numeric data

In [3]:
from sklearn.datasets import load_iris
iris = load_iris()

X = iris.data
y = iris.target


In [4]:
import pandas as pd
X2 = pd.DataFrame(X, columns = iris.feature_names)

In [5]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

knn.fit(X,y)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

#### Applying text to vector conversion on a smaller dataset

In [6]:
simpleTrain = ['call me tonight', 'call u later', 'cant talk rn...... busy! ']
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(simpleTrain)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [7]:
print vec.get_feature_names()

[u'busy', u'call', u'cant', u'later', u'me', u'rn', u'talk', u'tonight']


In [27]:
simpleTrainDTM = vec.transform(simpleTrain ) #conveting the training data to document term matrix
simpleTrainDTM

<3x8 sparse matrix of type '<type 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [28]:
simpleTrainDTM.toarray()

array([[0, 1, 0, 0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 1, 1, 0]])

In [29]:
pd.DataFrame(simpleTrainDTM.toarray(), columns = vec.get_feature_names())

,busy,call,cant,later,me,rn,talk,tonight
0,0,1,0,0,1,0,0,1
1,0,1,0,1,0,0,0,0
2,1,0,1,0,0,1,1,0


## Now we are going to build or spam or ham detector from real life Data
#### After understanding and application of the vectorization method to small data set let us include a large reallife data
#### stored in sms.tsv file.

In [11]:
sms = pd.read_table('sms.tsv', header =None, names = ['label', 'msg'])
sms.head()

/home/acer/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


,label,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [20]:
sms['labNum'] = sms.label.map({'ham':0 , 'spam':1})
sms.drop(['lableMark'], axis =1,inplace=True)
sms.head()

,label,msg,labNum
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [28]:
x = sms.msg
y = sms.labNum
#splitting into treainig and testing data
from sklearn.model_selection import train_test_split

xtrain , xtest , ytrain, ytest = train_test_split(x,y,random_state=1)

vect = CountVectorizer()
vect.fit(xtrain)
vectDTM = vect.transform(xtrain)

trainVect = pd.DataFrame(vectDTM.toarray(), columns = vect.get_feature_names())

In [34]:
#use the intial vocabulary for this and not a new vocabulary
vect2DTM = vect.transform(xtest)

testVect= pd.DataFrame(vect2DTM.toarray(), columns = vect.get_feature_names())


In [30]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

%time nb.fit(trainVect, ytrain)

CPU times: user 685 ms, sys: 413 ms, total: 1.1 s
Wall time: 1.13 s


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [35]:
ypred = nb.predict(testVect)  #testVect is the test data vectorized

In [37]:
from sklearn import metrics
print metrics.accuracy_score(ytest, ypred)

0.9885139985642498


In [38]:
metrics.confusion_matrix(ytest, ypred)

array([[1203,    5],
       [  11,  174]])

In [41]:
#using logestic regeresssion for this data

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
%time lr.fit(trainVect, ytrain)

ypred_lr = lr.predict(testVect)


CPU times: user 346 ms, sys: 191 ms, total: 537 ms
Wall time: 542 ms


In [ ]:
#as we can see that logestic regression is faster than naive bayes

In [40]:
print metrics.accuracy_score(ytest, ypred_lr)

0.9877961234745154


In [42]:
#depending on the machine you can switch easily between the naive bayes  and logesticregression

In [46]:
#trainig the complete data without split for finalizing the model.

vectf = CountVectorizer()
vectf.fit(x)
vectfDTM = vectf.transform(x)

trainFinal = pd.DataFrame(vectfDTM.toarray(), columns = vectf.get_feature_names())

In [47]:
#FINAL TRAINED MODELS
nb.fit(trainFinal,y)
lr.fit(trainFinal,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)